<a href="https://colab.research.google.com/github/GopalKrishna-India/ML_AI_for_TOF_Mapping/blob/main/BuildingTensorFlowModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install the earth engine API, Google Cloud API and authenticate

In [ ]:
!pip install earthengine-api
from google.colab import auth

auth.authenticate_user()

# Import the Earth Engine API and initialize it.
import ee

ee.Authenticate()
ee.Initialize()

     |████████████████████████████████| 153kB 4.8MB/s 
  Created wheel for earthengine-api: filename=earthengine_api-0.1.204-cp36-none-any.whl size=175146 sha256=1a454567277fda277e6af3046082884e8f3c9fa13caf46edec2e3c63943e211d
  Stored in directory: /root/.cache/pip/wheels/1c/84/12/072bd182af991b0f0b9c1599de80fa7b15179ba427106c5b17
Successfully built earthengine-api
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verificat

# Import the tensorflow library and import the training and testing data

In [ ]:
import tensorflow as tf

tf.enable_eager_execution()
print(tf.__version__)

trainFilePath = 'gs://yourbucket/Hanoi/trainFile.tfrecord.gz'
testFilePath = 'gs://yourbucket/Hanoi/testFile.tfrecord.gz'

print('Found training file.' if tf.gfile.Exists(trainFilePath)
    else 'No training file found.')

# Create a dataset from the TFRecord file in Cloud Storage.
trainDataset = tf.data.TFRecordDataset(trainFilePath, compression_type='GZIP')
# Print the first record to check.
print(iter(trainDataset).next())


# Set the label and feature names

In [ ]:
label = "land_class"

featureNames = list(['cb','blue','green','red','re1','re2','re3','nir','re4','waterVapor','cirrus','swir1','swir2',"ndvi"])
featureNames.append(label)

l = len(featureNames)
print(l)
featureNames = sorted(featureNames)
featureNames.append(label)

# List of fixed-length features, all of which are float32.
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in featureNames
]

# Dictionary with names as keys, features as values.
featuresDict = dict(zip(featureNames, columns))

print(featuresDict)

# Read a serialized example into the structure defined by featuresDict.

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.

  Read a serialized example into the structure defined by featuresDict.

  Args:
    example_proto: a serialized Example.

  Returns:
    A tuple of the predictors dictionary and the label, cast to an `int32`.
  """
  parsed_features = tf.io.parse_single_example(example_proto, featuresDict)
  labels = parsed_features.pop(label)
  return parsed_features, tf.cast(labels, tf.int32)

# Map the function over the dataset.
parsedDataset = trainDataset.map(parse_tfrecord, num_parallel_calls=5)

# Print the first parsed record to check.
print(iter(parsedDataset).next())

({'blue': <tf.Tensor: id=15005, shape=(1,), dtype=float32, numpy=array([0.1179], dtype=float32)>, 'cb': <tf.Tensor: id=15006, shape=(1,), dtype=float32, numpy=array([0.1476], dtype=float32)>, 'cirrus': <tf.Tensor: id=15007, shape=(1,), dtype=float32, numpy=array([0.0012], dtype=float32)>, 'green': <tf.Tensor: id=15008, shape=(1,), dtype=float32, numpy=array([0.0969], dtype=float32)>, 'ndvi': <tf.Tensor: id=15009, shape=(1,), dtype=float32, numpy=array([0.38778746], dtype=float32)>, 'nir': <tf.Tensor: id=15010, shape=(1,), dtype=float32, numpy=array([0.175], dtype=float32)>, 're1': <tf.Tensor: id=15011, shape=(1,), dtype=float32, numpy=array([0.1], dtype=float32)>, 're2': <tf.Tensor: id=15012, shape=(1,), dtype=float32, numpy=array([0.1391], dtype=float32)>, 're3': <tf.Tensor: id=15013, shape=(1,), dtype=float32, numpy=array([0.1529], dtype=float32)>, 're4': <tf.Tensor: id=15014, shape=(1,), dtype=float32, numpy=array([0.1648], dtype=float32)>, 'red': <tf.Tensor: id=15015, shape=(1,), d

# Build the model

In [ ]:
from tensorflow import keras


# How many classes there are in the model.
nClasses = 5

# Add features .
inputDataset = parsedDataset #.map(addFeatures)

# Keras requires inputs as a tuple.  Note that the inputs must be in the
# right shape.  Also note that to use the categorical_crossentropy loss,
# the label needs to be turned into a one-hot vector.
def toTuple(dict, label):
  #return tf.transpose(list(dict.values())), tf.one_hot(indices=label, depth=nClasses)
  return (tf.expand_dims(tf.transpose(list(dict.values())), 1),
          tf.expand_dims(tf.one_hot(indices=label, depth=nClasses), 1))

# Repeat the input dataset as many times as necessary in batches.
inputDataset = inputDataset.map(toTuple).shuffle(3000).batch(300).repeat()

# Define the layers in the model.
model = tf.keras.models.Sequential([
  tf.keras.layers.Input((None, None, l-1,)),
  tf.keras.layers.Conv2D(512, (1, 1), activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Conv2D(256, (1, 1), activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Conv2D(nClasses, (1, 1), activation=tf.nn.softmax)
])

# Compile the model with the specified loss function.
model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.005),#'adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Fit the model to the training data.
# Don't forget to specify `steps_per_epoch` when calling `fit` on a dataset.
training = model.fit(x=inputDataset, epochs=100,steps_per_epoch=10)

%pylab inline
plot(training.history['loss'],'x--')
plot(training.history['acc'], 'o--')

# Do the data validation using the test data

In [ ]:
testDataset = (
  tf.data.TFRecordDataset(testFilePath, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(toTuple)
    .batch(1)
)

model.evaluate(testDataset)

    151/Unknown - 2s 10ms/step - loss: 0.4941 - acc: 0.8146

[0.49410629941916157, 0.81456953]

# Save your model in your cloud bucket

In [ ]:
outputBucket = "yourbucket/Hanoi/"
MODEL_DIR = 'gs://' + outputBucket + '/model'
tf.contrib.saved_model.save_keras_model(model, MODEL_DIR)

# Make your model readable for the earthEngine

In [ ]:
from tensorflow.python.tools import saved_model_utils

meta_graph_def = saved_model_utils.get_meta_graph_def(MODEL_DIR, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
  input_name = v.name
  break

output_name = None
for k,v in outputs.items():
  output_name = v.name
  break

# Make a dictionary that maps Earth Engine outputs and inputs to
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: label}) + "'"

print(input_dict)
print(output_dict)

# Put the EEified model next to the trained model directory.
EEIFIED_DIR = 'gs://' + outputBucket + '/eeified'
PROJECT = 'servir-rlcms'

#print(EEIFIED_DIR)
# You need to set the project before using the model prepare command.
!earthengine set_project {PROJECT}
!earthengine model prepare --source_dir {MODEL_DIR} --dest_dir {EEIFIED_DIR} --input {input_dict} --output {output_dict}

# Push your model to the AI platform

In [ ]:
import time
MODEL_NAME = 'Hanoi'
VERSION_NAME = 'v' + str(int(time.time()))
print('Creating version: ' + VERSION_NAME)

#!gcloud ai-platform models create {MODEL_NAME} --project {PROJECT}
!gcloud ai-platform versions create {VERSION_NAME} \
  --project {PROJECT} \
  --model {MODEL_NAME} \
  --origin {EEIFIED_DIR} \
  --runtime-version=1.14 \
  --framework "TENSORFLOW" \
  --python-version=3.5

Creating version: v1571915575
